In [ ]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import h3
import folium
from folium.plugins import MarkerCluster


excel_file = "nfh_addresses.xlsx"
file_output_path = "weather_interpolation_locations.parquet"
file_output_excel = "weather_interpolation_locations.xlsx"

In [ ]:
resolution = 4 

In [ ]:
# Initialize geocoder
geolocator = Nominatim(user_agent="myGeocoder")

# Function to geocode address to latitude and longitude
def geocode_address(pc6, number, add_on):
    address = f"{pc6}, {number}{add_on}, Netherlands "
    location = geolocator.geocode(address)
    if location:
        return (location.latitude, location.longitude)
    else:
        return None

# Function to convert latitude and longitude to H3 index
def lat_lon_to_h3(lat, lon, resolution):
    return h3.geo_to_h3(lat, lon, resolution)

# Function to get the center of H3 cell
def h3_to_lat_lon(h3_index):
    lat, lon = h3.h3_to_geo(h3_index)
    return lat, lon
    
# Main function to convert addresses to H3
def addresses_to_h3(addresses, resolution=4):
    results = []
    for index, row in addresses.iterrows():
        address = f'{row["PC6"]}, {row["home_nr"]}{row["home_nr_add_on"]}, Netherlands'
        lat_lon = geocode_address(row['PC6'], row['home_nr'], row['home_nr_add_on'])
        if lat_lon:
            h3_index = lat_lon_to_h3(lat_lon[0], lat_lon[1], resolution)
            h3_center = h3_to_lat_lon(h3_index)
            results.append({
                'Address': address,
                'Geocoded Lat': lat_lon[0],
                'Geocoded Lon': lat_lon[1],
                'H3 Cell ID': h3_index,
                'H3 Center Lat': h3_center[0],
                'H3 Center Lon': h3_center[1]
            })
    return results

# Function to add weather_lat and weather_lon to DataFrame
def add_weather_coordinates(addresses_df, h3_coordinates):
    weather_lat = [coord['H3 Center Lat'] for coord in h3_coordinates]
    weather_lon = [coord['H3 Center Lon'] for coord in h3_coordinates]
    addresses_df['weather_lat'] = weather_lat
    addresses_df['weather_lon'] = weather_lon
    return addresses_df



In [ ]:
# Read addresses from Excel file
addresses_df = pd.read_excel(excel_file)

In [ ]:
# Convert NaN values in 'home_nr_add_on' to empty strings
addresses_df['home_nr_add_on'] = addresses_df['home_nr_add_on'].replace(np.nan, '')


In [ ]:
addresses_df.info()

In [ ]:
# addresses_df

In [ ]:
# Convert addresses to H3 coordinates
h3_coordinates  = addresses_to_h3(addresses_df, resolution)


In [ ]:
# h3_coordinates 

In [ ]:
# Create a folium map centered around the first address
map_center = h3_coordinates[0]['Geocoded Lat'], h3_coordinates[0]['Geocoded Lon']
mymap = folium.Map(location=map_center, zoom_start=12)

# Add markers for home addresses
marker_cluster = MarkerCluster().add_to(mymap)
# for result in h3_coordinates:
#     folium.Marker([result['Geocoded Lat'], result['Geocoded Lon']], popup=result['Address']).add_to(marker_cluster)

# Add H3 cells and center points
for result in h3_coordinates:
    lat_lon = (result['H3 Center Lat'], result['H3 Center Lon'])
    folium.Marker(lat_lon, icon=folium.Icon(color='red', icon='cross')).add_to(mymap)
    hexagon = h3.h3_to_geo_boundary(result['H3 Cell ID'])
    folium.Polygon(locations=hexagon, color='blue', fill=True, fill_opacity=0.2).add_to(mymap)

# Save map to HTML file
mymap.save("map_with_h3_cells.html")

# Display map
mymap

In [ ]:
# Add weather coordinates to the DataFrame
df = add_weather_coordinates(addresses_df, h3_coordinates)

In [ ]:
# df

In [ ]:
# Read the mapping DataFrame from the Excel file
mapping_df = pd.read_excel('pseudonym_id_student_mapping.xlsx')

In [ ]:
mapping_df

In [ ]:
df.info()

In [ ]:
# Rename 'pseudonym' column to 'id'
df.rename(columns={'pseudonym': 'id'}, inplace=True)

In [ ]:
df.id.unique()

In [ ]:
# # Convert the mapping DataFrame to a dictionary with 'id' as key and 'random_id' as value
# id_mapping_dict = dict(zip(mapping_df['id'], mapping_df['random_id']))

# # Replace 'id' values in the DataFrame using the dictionary
# df['id'] = df['id'].map(id_mapping_dict).astype('Int64')

In [ ]:
df.id.unique()

In [ ]:
df.drop(columns=['PC6', 'home_nr', 'home_nr_add_on'], inplace=True)

In [ ]:
df = df.dropna(subset=['id'])

In [ ]:
df

In [ ]:
%%time 
df.to_parquet(file_output_path, index=True, engine='pyarrow')

In [ ]:
# Save the DataFrame back to Excel
df.to_excel(file_output_excel, index=False)